Test model llama3

In [1]:
%pip install -q python-dotenv~=1.0.1 lamini==3.0.5

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
_ = load_dotenv() #load environmental variable LAMINI_API_KEY with key from .env file

import lamini
# Token access to https://app.lamini.ai/tune/
lamini.api_key = "ad3d62932bc25b48963dd3413dd94cb3414dd650fc47f735704badeb56cdf9ca" # Active API Tokens

In [3]:
# Load the llm3 model
llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3-8B-Instruct")

In [4]:
# Create prompt with template of llm3 model
prompt = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Please write a graduate card for my good friend Mạnh Tiền\
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

# Return result of question by llm3 model
result = llm.generate(prompt, max_new_tokens=200)
print(result)

Here's a possible graduate card for your friend Mạnh Tiền:

"Dear Mạnh Tiền,

Huge congratulations on your graduation!

You've worked hard to achieve this milestone, and it's an incredible accomplishment. I'm so proud of you for pushing through challenges and staying committed to your goals.

As you move forward, remember that this is just the beginning of an exciting new chapter in your life. You have so much to look forward to, and I have no doubt that you'll make the most of every opportunity that comes your way.

Here's to many more adventures, successes, and memories to come! Keep shining your light and making a difference in the world.

Best wishes,
[Your Name]"

Feel free to modify it as you see fit to make it more personal and special for your friend Mạnh Tiền!


In [9]:
prompt2 = (
  "<|begin_of_text|>"  # Start of prompt
  "<|start_header_id|>system<|end_header_id|>\n\n"  #  header - system
  "You are a helpful assistant."  # system prompt
  "<|eot_id|>" # end of turn
  "<|start_header_id|>user<|end_header_id|>\n\n" # header - user
  "Please write a graduate card for my good friend Mạnh Tiền"
  "<|eot_id|>" # end of turn
  "<|start_header_id|>assistant<|end_header_id|>\n\n" # header - assistant
)

prompt = prompt2

# Return result of question by llm3 model
result = llm.generate(prompt, max_new_tokens=200)
print(result)

Here's a birthday card message for your friend Mạnh Tiền:

"Dear Mạnh Tiền,

Happy birthday to an amazing friend like you! On your special day, I want to take a moment to express my gratitude for the incredible memories we've shared and the laughter we've enjoyed together. You bring so much joy and positivity to those around you, and I feel lucky to have you in my life.

Here's to another year of adventure, good health, and making more unforgettable memories together! May your birthday be as bright and wonderful as you are.

Wishing you a day filled with love, laughter, and all your favorite things.

Best regards, [Your Name]"

Feel free to modify it as you see fit!


In [11]:
from utils_llm import make_llama_3_prompt

system_prompt = "You are a helpful assistant." # system prompt
user_prompt = "Please write a graduate card for my good friend Mạnh Tiền"

prompt3 = make_llama_3_prompt(user_prompt, system_prompt)
prompt = prompt3

# Return result of question by llm3 model
result = llm.generate(prompt, max_new_tokens=200)
print(result)

Here's a possible graduate card for your friend Mạnh Tiền:

"Dear Mạnh Tiền,

Huge congratulations on your graduation!

You've worked hard to achieve this milestone, and it's an incredible accomplishment. I'm so proud of you for pushing through challenges and staying committed to your goals.

As you move forward, remember that this is just the beginning of an exciting new chapter in your life. You have so much to look forward to, and I have no doubt that you'll make the most of every opportunity that comes your way.

Here's to many more adventures, successes, and memories to come! Keep shining your light and making a difference in the world.

Best wishes,
[Your Name]"

Feel free to modify it as you see fit to make it more personal and special for your friend Mạnh Tiền!


In [13]:
user_prompt = "Tell me a joke about birthday cake"
prompt = make_llama_3_prompt(user_prompt)

result = llm.generate(prompt, max_new_tokens=200)
print(result)

Why was the birthday cake in a bad mood?

Because it was feeling crumby!


Create a SQL Agent

In [29]:
import logging
import sqlite3
import pandas as pd
from utils_llm import setup_logging, get_schema
import random

In [30]:
logger = logging.getLogger(__name__)
logger.info("Logger has been set up!")

engine = sqlite3.connect("./nba_roster.db")
# # Create a cursor object using the connection
# cursor = engine.cursor()

# # Sample data template
# teams = ['Los Angeles Lakers', 'Golden State Warriors', 'Brooklyn Nets', 'Milwaukee Bucks', 'Dallas Mavericks']
# names = ['Player ' + str(i) for i in range(1, 101)]
# positions = ['PG', 'SG', 'SF', 'PF', 'C']
# colleges = ['College A', 'College B', 'College C', 'College D', 'College E']

# # Function to generate random salary
# def generate_salary():
#     return f"${random.randint(1000000, 50000000):,}"

# # Generating 100 rows of data
# nba_data = [
#     (
#         random.choice(teams),
#         name,
#         str(random.randint(0, 99)),  # Jersey number
#         random.choice(positions),
#         random.randint(19, 40),  # Age
#         f'{random.randint(6, 7)}-{random.randint(0, 11)}',  # Height (e.g., 6-9)
#         str(random.randint(190, 260)),  # Weight
#         random.choice(colleges),
#         generate_salary()
#     )
#     for name in names
# ]

# # Insert the generated data into the nba_roster table
# cursor.executemany('''
# INSERT INTO nba_roster (team, name, jersey, pos, age, ht, wt, college, salary)
# VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
# ''', nba_data)

# # Commit the changes
# engine.commit()
# logger.info(f"Connected to the database at './film_database.db': {engine}")

setup_logging()

In [31]:
user = """Who is the highest paid NBA player?"""

system = f"""You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
{get_schema()}

Write a sqlite query to answer the following question. Follow instructions exactly"""

prompt = make_llama_3_prompt(user, system)

print(llm.generate(prompt, max_new_tokens=200))

To answer this question, we can use the following SQLite query:

```sql
SELECT NAME, SALARY
FROM nba_roster
WHERE SALARY!= '--'
ORDER BY CAST(SALARY AS REAL) DESC
LIMIT 1;
```

This query first filters out the rows where the salary is '--' (i.e., the players who don't have a salary listed). Then, it orders the remaining rows by the salary in descending order (highest to lowest). Finally, it limits the result to the top 1 row, which corresponds to the highest paid NBA player.


In [34]:
# We'd like to be able to get just SQL output so we don't have to parse the query from the model response. For this we can use structured output.
result = llm.generate(prompt, output_type={"sqlite_query": "str"}, max_new_tokens=200)
print(result)

# Query result
df = pd.read_sql(result['sqlite_query'], con=engine)
df

{'sqlite_query': "SELECT * FROM nba_roster WHERE SALARY!= '--' ORDER BY CAST(SALARY AS REAL) DESC LIMIT 1"}


,id,team,name,jersey,pos,age,ht,wt,college,salary
0,1,Brooklyn Nets,Player 1,18,SF,31,7-2,211,College D,"$17,565,125"
